In [ ]:
# --- 必要库导入 ---
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from pandas.api.types import is_numeric_dtype
import matplotlib.pyplot as plt
import random
import math
import os # 新增导入os库，用于处理文件路径

# ========== 0. 全局设置 ==========
def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark      = False
set_seed(42)
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']; plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-whitegrid')

# ========== 1. 基流分割 ==========
def eckhardt_filter(series, BFI_max=0.80, a=0.98):
    Q = series.values
    b = np.zeros_like(Q); b[0] = Q[0] * BFI_max
    n1, n2, den = (1-BFI_max)*a, (1-a)*BFI_max, 1-a*BFI_max
    for i in range(1, len(Q)):
        b_t = (n1 * b[i-1] + n2 * Q[i]) / den
        b[i] = min(b_t, Q[i])
    return b, Q - b

# ========== 2. 数据加载 ==========
# !!! 注意：请将这里的路径修改为您自己电脑上的实际文件路径 !!!
try:
    df = pd.read_excel(r"D:\dataset\ChinaMet_洛河区域汇总\14-21.xlsx")
except FileNotFoundError:
    print("错误：无法在 D:\\dataset\\ChinaMet_洛河区域汇总\\14-21.xlsx 找到数据文件。")
    print("请在代码第49行修改为您的实际文件路径后再次运行。")
    # 如果找不到文件，创建一个空的DataFrame以避免后续代码报错
    df = pd.DataFrame() 

if not df.empty:
    df['Time'] = pd.to_datetime(df['Time']); df.set_index('Time', inplace=True)
    df['Baseflow'], df['Quickflow'] = eckhardt_filter(df['Runoff'])
    # *** 修改 1/2：不再删除 'Runoff' 列，将其保留用于最终输出 ***
    # df.drop(columns='Runoff', inplace=True) # 此行被注释掉

    # ========== 2.5 高级特征工程 ==========
    print("\n--- 开始进行高级特征工程 ---")
    df['pre_sum_3d']  = df['pre'].rolling(3, 1).sum()
    df['pre_sum_7d']  = df['pre'].rolling(7, 1).sum()
    df['pre_sum_14d'] = df['pre'].rolling(14, 1).sum()
    df['pre_sum_30d'] = df['pre'].rolling(30, 1).sum()
    df['SMrz_mean_14d'] = df['SMrz'].rolling(14, 1).mean()
    df['rhu_mean_14d']  = df['rhu'].rolling(14, 1).mean()
    df['petm_sum_14d']  = df['petm'].rolling(14, 1).sum()
    is_rain = df['pre'] > 1.0
    days_since_rain = is_rain.cumsum()
    df['days_since_last_rain'] = days_since_rain.sub(days_since_rain.where(is_rain).ffill().fillna(0)).astype(int)
    df['pre_std_7d'] = df['pre'].rolling(7, 1).std().fillna(0)
    k = 0.9
    api_values = np.zeros(len(df))
    pre_values = df['pre'].values
    for i in range(1, len(df)):
        api_values[i] = k * api_values[i-1] + pre_values[i]
    df['api'] = api_values
    print("新增季节性特征和交互特征...")
    df['dayofyear'] = df.index.dayofyear
    df['sin_doy'] = np.sin(2 * np.pi * df['dayofyear'] / 366)
    df['cos_doy'] = np.cos(2 * np.pi * df['dayofyear'] / 366)
    df.drop(columns='dayofyear', inplace=True)
    df['api_x_pre_sum_3d'] = df['api'] * df['pre_sum_3d']
    df.dropna(inplace=True)
    print(f"特征工程完成，处理NaN后数据从 {df.index[0]} 开始。")

# ========== 3. 数据预处理函数 ==========
def prepare_data_for_model(df_org, target, non_lag_feats, max_lag=7, corr_thr=0.3, look_back=30, use_log=False, std_target=False):
    data = df_org.copy()
    if use_log: data[target] = np.log1p(data[target])
    feats_to_lag = [c for c in df_org.columns if c not in non_lag_feats and is_numeric_dtype(df_org[c])]
    for c in feats_to_lag:
        for k in range(1, max_lag+1):
            data[f"{c}_lag{k}"] = df_org[c].shift(k)
    # 此处会自动丢弃原始的'Runoff'列(因为它不是target)，所以不会影响模型训练
    data.drop(columns=[c for c in feats_to_lag if c != target], inplace=True)
    data.dropna(inplace=True)
    engineered = ['pre_sum_3d','pre_sum_7d','pre_sum_14d','pre_sum_30d','SMrz_mean_14d','rhu_mean_14d','petm_sum_14d','days_since_last_rain','pre_std_7d','api', 'sin_doy', 'cos_doy', 'api_x_pre_sum_3d']
    sel = data.corr()[target].abs().pipe(lambda s: s[s>=corr_thr]).index.tolist()
    if target not in sel: sel.append(target)
    for f in engineered:
        if f in data and f not in sel: sel.append(f)
    data = data[sel]
    X_raw, y_raw = data.drop(columns=target), data[[target]]
    scaler_X = MinMaxScaler().fit(X_raw); X = scaler_X.transform(X_raw)
    scaler_y = StandardScaler() if std_target else MinMaxScaler()
    y = scaler_y.fit_transform(y_raw).flatten()
    arr = np.hstack([X, y.reshape(-1,1)])
    tgt_idx = arr.shape[1]-1
    X_seq, y_seq = [], []
    for i in range(len(arr)-look_back):
        X_seq.append(arr[i:i+look_back]); y_seq.append(arr[i+look_back, tgt_idx])
    X_seq = torch.tensor(np.array(X_seq), dtype=torch.float32)
    y_seq = torch.tensor(np.array(y_seq), dtype=torch.float32).view(-1,1)
    n = len(X_seq); n_train, n_val = int(n*0.7), int(n*0.15)
    X_train, y_train = X_seq[:n_train], y_seq[:n_train]
    X_val,   y_val   = X_seq[n_train:n_train+n_val], y_seq[n_train:n_train+n_val]
    X_test,  y_test  = X_seq[n_train+n_val:], y_seq[n_train+n_val:]
    train_loader = DataLoader(TensorDataset(X_train, y_train), 64, shuffle=True)
    val_loader   = DataLoader(TensorDataset(X_val, y_val), 64, shuffle=False)
    test_loader  = DataLoader(TensorDataset(X_test, y_test), 64, shuffle=False)
    train_dates = data.index[look_back : look_back + n_train]
    test_dates = data.index[look_back + n_train + n_val:]
    return (train_loader, val_loader, test_loader, scaler_y, X_train.shape[2], y_test.numpy(), y_train, train_dates, test_dates)

# ========== 4. 数据准备 ==========
if not df.empty:
    look_back = 30
    today_feats = [c for c in ['SMrz','SMs','petm','pre','pres','rhu','tepmax','tepmin','tepmean','wind', 'sin_doy', 'cos_doy', 'api_x_pre_sum_3d'] if c in df.columns]
    base_dl  = prepare_data_for_model(df,'Baseflow', today_feats,3,0.3,look_back,False,False)
    (train_b,val_b,test_b,sc_b,dim_b,ytest_b,_,_,_) = base_dl
    quick_dl = prepare_data_for_model(df,'Quickflow', today_feats,3,0.2,look_back,True,True)
    (train_q,val_q,test_q,sc_q,dim_q,ytest_q,y_train_q, train_dates_q, test_dates_q) = quick_dl

# ========== 5. 模型 & 连续加权损失 ==========
class LSTMHead(nn.Module):
    def __init__(self, inp, hid, layers, outp, drop):
        super().__init__()
        self.lstm = nn.LSTM(inp, hid, layers, batch_first=True, dropout=drop if layers > 1 else 0)
        self.drop, self.fc = nn.Dropout(drop), nn.Linear(hid, outp)
    def forward(self, x):
        _, (h, _) = self.lstm(x)
        return self.fc(self.drop(h[-1]))

class CNNGRU(nn.Module):
    def __init__(self, input_dim, cnn_out_channels, cnn_kernel_size, gru_hidden_dim, gru_num_layers, output_dim, dropout_rate):
        super(CNNGRU, self).__init__()
        self.conv1d = nn.Conv1d(in_channels=input_dim, 
                                out_channels=cnn_out_channels, 
                                kernel_size=cnn_kernel_size, 
                                padding=(cnn_kernel_size - 1) // 2)
        self.gru = nn.GRU(cnn_out_channels, gru_hidden_dim, gru_num_layers,
                          batch_first=True, dropout=dropout_rate if gru_num_layers > 1 else 0)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(gru_hidden_dim, output_dim)
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1d(x)
        x = F.relu(x)
        x = x.permute(0, 2, 1)
        gru_out, _ = self.gru(x)
        last_time_step_out = gru_out[:, -1, :]
        output = self.fc(self.dropout(last_time_step_out))
        return output

class AsymmetricWeightedMSE(nn.Module):
    def __init__(self, q_mid, q_high, q_extreme, 
                 w_mid=5.0, w_high=15.0, w_extreme=30.0, 
                 overestimation_penalty=0.7):
        super().__init__()
        self.q_mid, self.q_high, self.q_extreme = q_mid, q_high, q_extreme
        self.w_mid, self.w_high, self.w_extreme = w_mid, w_high, w_extreme
        self.overestimation_penalty = overestimation_penalty
        self.mse = nn.MSELoss(reduction='none')
    def forward(self, preds, targets):
        base_weights = torch.ones_like(targets, device=targets.device)
        base_weights[targets >= self.q_mid] = self.w_mid
        base_weights[targets >= self.q_high] = self.w_high
        base_weights[targets >= self.q_extreme] = self.w_extreme
        loss = self.mse(preds, targets)
        weighted_loss = base_weights * loss
        asym_weights = torch.ones_like(targets, device=targets.device)
        asym_weights[preds > targets] = self.overestimation_penalty
        final_loss = torch.mean(asym_weights * weighted_loss)
        return final_loss

# ========== 6. 训练器 ==========
def fit(model, loaders, crit, opt, sched, ne=150, patience=30, name=''):
    tr,vl,best,bad = [],[],1e9,0; best_state=None
    print(f"\n--- 开始训练模型: {name} (早停={patience}) ---")
    dev = next(model.parameters()).device
    for ep in range(1,ne+1):
        model.train(); s=0
        for x,y in loaders[0]:
            x,y=x.to(dev),y.to(dev); opt.zero_grad()
            loss=crit(model(x),y); loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(),1.0); opt.step(); s+=loss.item()
        tr.append(s/len(loaders[0]))
        model.eval(); s=0
        with torch.no_grad():
            for x,y in loaders[1]:
                s += crit(model(x.to(dev)), y.to(dev)).item()
        v = s/len(loaders[1]); vl.append(v); sched.step(v)
        if v<best: best,bad,best_state = v,0,model.state_dict()
        else: bad+=1
        if ep%10==0: print(f"{name} Ep[{ep:03d}/{ne}]  tr={tr[-1]:.4e}  vl={v:.4e}")
        if bad>=patience: print(f"→ 早停于 Ep {ep}"); break
    model.load_state_dict(best_state); model.eval(); outs=[]
    with torch.no_grad():
        for x,_ in loaders[2]:
            outs.extend(model(x.to(dev)).cpu().numpy())
    return np.array(outs)

# ========== 7. 实例化 & 训练 ==========
if not df.empty:
    dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    m_base = LSTMHead(dim_b, 128, 2, 1, 0.2).to(dev)
    opt_b  = torch.optim.Adam(m_base.parameters(), 2e-3, weight_decay=1e-5)
    sch_b  = torch.optim.lr_scheduler.ReduceLROnPlateau(opt_b,'min',factor=0.7,patience=10)
    pred_b = fit(m_base,(train_b,val_b,test_b),nn.MSELoss(),opt_b,sch_b, name='Baseflow_LSTM')
    q_mid = torch.quantile(y_train_q, 0.85).item()
    q_high = torch.quantile(y_train_q, 0.95).item()
    q_extreme = torch.quantile(y_train_q, 0.99).item()
    print(f"\nQuickflow 微雕分段阈值: q85={q_mid:.4f}, q95={q_high:.4f}, q99={q_extreme:.4f}")
    m_quick = CNNGRU(input_dim=dim_q, cnn_out_channels=64, cnn_kernel_size=3,
                     gru_hidden_dim=128, gru_num_layers=2, output_dim=1, dropout_rate=0.4).to(dev)
    opt_q   = torch.optim.Adam(m_quick.parameters(), 5e-4, weight_decay=1e-5)
    sch_q   = torch.optim.lr_scheduler.ReduceLROnPlateau(opt_q,'min',factor=0.7,patience=10)
    crit_q = AsymmetricWeightedMSE(q_mid, q_high, q_extreme,
                                   w_mid=5.0, w_high=15.0, w_extreme=35.0,
                                   overestimation_penalty=0.7).to(dev)
    pred_q  = fit(m_quick,(train_q,val_q,test_q),crit_q,opt_q,sch_q, name='Quickflow_CNNGRU', patience=50)

# ========== 8. 逆变换 & 评估 ==========
if not df.empty:
    def inv_transform(scaler, data):
        if data.ndim == 1: data = data.reshape(-1, 1)
        return scaler.inverse_transform(data)
    def nash_sutcliffe(obs, sim):
        return 1 - np.sum((obs - sim) ** 2) / np.sum((obs - np.mean(obs)) ** 2)
    
    y_b   = inv_transform(sc_b, ytest_b)
    p_b   = inv_transform(sc_b, pred_b)
    y_q_l = inv_transform(sc_q, ytest_q)
    p_q_l = inv_transform(sc_q, pred_q)
    y_q, p_q = np.expm1(y_q_l), np.expm1(p_q_l)
    p_b[p_b<0]=0; p_q[p_q<0]=0
    
    print("\n--- 开始独立评估各个模型组件 ---")
    def evaluate_component(y_true, y_pred, name=''):
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        nse = nash_sutcliffe(y_true.flatten(), y_pred.flatten())
        print(f"\n--- {name} 模型性能 ---")
        print(f"RMSE = {rmse:.4f}; MAE = {mae:.4f}; R² = {r2:.4f}; NSE = {nse:.4f}")
        return nse
    
    nse_b = evaluate_component(y_b, p_b, "基流(Baseflow)")
    nse_q = evaluate_component(y_q, p_q, "快速流(Quickflow)")
    y_all, p_all = y_b + y_q, p_b + p_q
    rmse = np.sqrt(mean_squared_error(y_all, p_all))
    mae  = mean_absolute_error(y_all, p_all)
    r2   = r2_score(y_all, p_all)
    nse = nash_sutcliffe(y_all.flatten(), p_all.flatten())
    print(f"\n--- 最终组合模型在测试集上的性能 ---")
    print(f"RMSE = {rmse:.4f} ; MAE = {mae:.4f} ; R² = {r2:.4f} ; NSE = {nse:.4f}")

# ========== 9. 高级可视化 ==========
if not df.empty:
    import matplotlib.dates as mdates
    plt.rc('font', family='Microsoft YaHei', size=10)
    plt.rc('axes', unicode_minus=False)
    plt.style.use('ggplot')
    final_dates = test_dates_q
    min_len = len(final_dates)
    p_all, y_all = p_all[:min_len], y_all[:min_len]
    fig, ax = plt.subplots(figsize=(15, 6))
    ax.plot(final_dates, y_all, 'b-', linewidth=1.8, label='真实总径流')
    ax.plot(final_dates, p_all, 'r-', linewidth=1.5, alpha=0.9, label='预测总径流')
    ax.set_title(f'总径流预测结果 (RMSE={rmse:.2f}, NSE={nse:.3f})', fontsize=14, pad=12)
    ax.set_xlabel('日期', fontsize=10); ax.set_ylabel('径流量 (mm)', fontsize=10)
    ax.legend(loc='upper right', frameon=True, shadow=True)
    ax.grid(True, linestyle='--', alpha=0.7)
    textstr = f'RMSE: {rmse:.2f}\nMAE: {mae:.2f}\nNSE: {nse:.3f}\nR²: {r2:.3f}'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.02, 0.95, textstr, transform=ax.transAxes, fontsize=10, verticalalignment='top', bbox=props)
    plt.tight_layout(); plt.show()
    
    print("\n--- 生成并排的可视化诊断图 ---")
    fig, axes = plt.subplots(3, 1, figsize=(15, 12), sharex=True)
    y_b_plot = y_b[:min_len]; p_b_plot = p_b[:min_len]
    y_q_plot = y_q[:min_len]; p_q_plot = p_q[:min_len]
    axes[0].plot(final_dates, y_all, 'b-', label='真实总径流')
    axes[0].plot(final_dates, p_all, 'r-', alpha=0.9, label='预测总径流')
    axes[0].set_title(f'总径流 (Overall) - NSE: {nse:.3f}', fontsize=14)
    axes[0].set_ylabel('径流量 (mm)'); axes[0].legend(); axes[0].grid(True, linestyle='--', alpha=0.7)
    axes[1].plot(final_dates, y_b_plot, 'b-', label='真实基流')
    axes[1].plot(final_dates, p_b_plot, 'r-', alpha=0.9, label='预测基流')
    axes[1].set_title(f'基流 (Baseflow) - NSE: {nse_b:.3f}', fontsize=14)
    axes[1].set_ylabel('径流量 (mm)'); axes[1].legend(); axes[1].grid(True, linestyle='--', alpha=0.7)
    axes[2].plot(final_dates, y_q_plot, 'b-', label='真实快速流')
    axes[2].plot(final_dates, p_q_plot, 'r-', alpha=0.9, label='预测快速流')
    axes[2].set_title(f'快速流 (Quickflow) - NSE: {nse_q:.3f}', fontsize=14)
    axes[2].set_ylabel('径流量 (mm)'); axes[2].legend(); axes[2].grid(True, linestyle='--', alpha=0.7)
    fig.autofmt_xdate(); plt.xlabel('日期'); plt.tight_layout(); plt.show()

# ========== 10. 洪峰期靶向评估 ==========
if not df.empty:
    print("\n--- 洪峰期靶向评估 ---")
    flood_thr = np.percentile(y_all, 95)
    idx_flood = y_all.flatten() > flood_thr
    if idx_flood.any():
        flood_dates = np.array(final_dates)[idx_flood]
        act_flood = y_all.flatten()[idx_flood]
        pre_flood = p_all.flatten()[idx_flood]
        flood_rmse = np.sqrt(mean_squared_error(act_flood, pre_flood))
        flood_mae  = mean_absolute_error(act_flood, pre_flood)
        flood_r2   = r2_score(act_flood, pre_flood)
        flood_nse  = nash_sutcliffe(act_flood, pre_flood)
        print(f"阈值：Total Runoff > {flood_thr:.2f}")
        print(f"Flood-RMSE = {flood_rmse:.4f} ; Flood-MAE = {flood_mae:.4f}")   
        print(f"Flood-R² = {flood_r2:.4f} ; Flood-NSE = {flood_nse:.4f}")
    else:
        print("测试集未检出超过 95% 分位的洪峰。")



# ========== 11. 保存预测结果到Excel ==========
if not df.empty:
    print("\n--- 开始保存预测结果到Excel ---")
    
    # 创建结果DataFrame
    results_df = pd.DataFrame({
        'time': test_dates_q,  # 使用测试集对应的时间索引
        'prediction': p_all.flatten(),  # 组合预测结果（基流+快速流）
        'true': y_all.flatten()  # 组合真实值（基流+快速流）
    })
    
    # 确保保存路径存在
    save_path = r"C:\Users\李\Desktop\论文\分析3"
    os.makedirs(save_path, exist_ok=True)
    # 计算并显示一些基本统计信息
    print("\n保存数据的统计信息:")
    print(f"预测值范围: {results_df['prediction'].min():.4f} - {results_df['prediction'].max():.4f}")
    print(f"真实值范围: {results_df['true'].min():.4f} - {results_df['true'].max():.4f}")
    print(f"时间范围: {results_df['time'].min()} 到 {results_df['time'].max()}")
    
    # 可选：同时保存分解后的各个组件
    detailed_results_df = pd.DataFrame({
        'time': test_dates_q,
        'baseflow_prediction': p_b.flatten(),
        'baseflow_true': y_b.flatten(),
        'quickflow_prediction': p_q.flatten(),
        'quickflow_true': y_q.flatten(),
        'total_prediction': p_all.flatten(),
        'total_true': y_all.flatten()
    })
    
    # 保存详细分解结果
    detailed_excel_path = os.path.join(save_path, "LSTM-CNN+GRU （分解）_详细.xlsx")
    detailed_results_df.to_excel(detailed_excel_path, index=False)
    print(f"\n详细分解结果已保存到: {detailed_excel_path}")
    
else:
    print("数据为空，无法保存结果。")
